## Estadistica Aplicada
Sesión 13-14 - 14-18 septiembre

Jorge Sebastian Aguirre Jimenez 1947870

G41

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from lifelines import CoxPHFitter
import seaborn as sns

In [3]:
df_cancer=pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/cancer.csv')

In [5]:
df_cancer

,inst,time,status,age,sex,ph.ecog,ph.karno,pat.karno,meal.cal,wt.loss
0,3.0,306,2,74,1,1.0,90.0,100.0,1175.0,NaN
1,3.0,455,2,68,1,0.0,90.0,90.0,1225.0,15.0
2,3.0,1010,1,56,1,0.0,90.0,90.0,NaN,15.0
3,5.0,210,2,57,1,1.0,90.0,60.0,1150.0,11.0
4,1.0,883,2,60,1,0.0,100.0,90.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
223,1.0,188,1,77,1,1.0,80.0,60.0,NaN,3.0
224,13.0,191,1,39,1,0.0,90.0,90.0,2350.0,-5.0
225,32.0,105,1,75,2,2.0,60.0,70.0,1025.0,5.0
226,6.0,174,1,66,1,1.0,90.0,100.0,1075.0,1.0


In [6]:
## Limpiar la base y calcular estadísticas descriptivas
df_cancer.head()

,inst,time,status,age,sex,ph.ecog,ph.karno,pat.karno,meal.cal,wt.loss
0,3.0,306,2,74,1,1.0,90.0,100.0,1175.0,NaN
1,3.0,455,2,68,1,0.0,90.0,90.0,1225.0,15.0
2,3.0,1010,1,56,1,0.0,90.0,90.0,NaN,15.0
3,5.0,210,2,57,1,1.0,90.0,60.0,1150.0,11.0
4,1.0,883,2,60,1,0.0,100.0,90.0,NaN,0.0


Los datos constan de 228 observaciones y 10 variables/columnas. La descripción de las variables es la siguiente:
inst: 

código de institución\
**time (d1)**: tiempo de supervivencia en días\
**status (d2)**: estado de censura 1 = censurado, 2 = muerto\
**age (i1)**: Edad en años\
**sex (i2)**: Masculino = 1 Femenino = 2\
**ph.ecog (i3)**: puntuación de rendimiento ECOG según la calificación del médico. 0 = asintomático, 1 = sintomático pero completamente ambulatorio, 2 = en cama <50% del día, 3 = en cama > 50% del día pero no encamado, 4 = encamado\
**ph.karno (i4)**: puntuación de desempeño de Karnofsky (mala = 0; buena = 100) calificada por el médico\
**pat.karno (i4)**: puntuación de rendimiento de Karnofsky según la calificación del paciente\
**meal.cal (i5)**: Calorías consumidas en las comidas\
**wt.loss (i6)**: Pérdida de peso en los últimos seis meses

tiene menú contextual

In [7]:
## hacerlas dummies
df_cancer['status']=df_cancer['status']-1

## status 1: fallecido, 0: censurados
## sex 1: mujeres, 0: hombres


In [8]:
## vamos a borrar la variable inst
df_cancer=df_cancer.drop('inst', axis=1)

In [9]:
## vamos a verificar que no existan missing values
df_cancer.isnull().sum()

time          0
status        0
age           0
sex           0
ph.ecog       1
ph.karno      1
pat.karno     3
meal.cal     47
wt.loss      14
dtype: int64

In [10]:
## imputar valores / eliminarlos de la base
df_cancer['ph.karno']=df_cancer['pat.karno'].fillna(df_cancer['ph.karno'].mean())
df_cancer['pat.karno']=df_cancer['pat.karno'].fillna(df_cancer['ph.karno'].mean())
df_cancer['meal.cal']=df_cancer['meal.cal'].fillna(df_cancer['meal.cal'].mean())
media_loss=df_cancer['wt.loss'].mean()
df_cancer['wt.loss']=df_cancer['wt.loss'].fillna(media_loss)

In [11]:
## cuantas observaciones tenemos
df_cancer.count()

time         228
status       228
age          228
sex          228
ph.ecog      227
ph.karno     228
pat.karno    228
meal.cal     228
wt.loss      228
dtype: int64

In [12]:
df_cancer=df_cancer.dropna()

In [13]:
df_cancer.isnull().sum()

time         0
status       0
age          0
sex          0
ph.ecog      0
ph.karno     0
pat.karno    0
meal.cal     0
wt.loss      0
dtype: int64

In [4]:
## verificar que variables son numericas
df_cancer.dtypes
## Sesion 14 al 18 de septiembre

inst         float64
time           int64
status         int64
age            int64
sex            int64
ph.ecog      float64
ph.karno     float64
pat.karno    float64
meal.cal     float64
wt.loss      float64
dtype: object

In [14]:
## recodificar ph.ecog como entero
df_cancer['ph.ecog']=df_cancer['ph.ecog'].astype('int64')

In [15]:
### Modelo de Cox (libreria lifelines) solo admite variables continuas y dummies (0,1)
### En lifelines las variables categoricas se tienen que convertir en dummies

In [16]:
dummies=pd.get_dummies(df_cancer['ph.ecog'],prefix='ecog').astype(int)

In [17]:
dummies

,ecog_0,ecog_1,ecog_2,ecog_3
0,0,1,0,0
1,1,0,0,0
2,1,0,0,0
3,0,1,0,0
4,1,0,0,0
...,...,...,...,...
223,0,1,0,0
224,1,0,0,0
225,0,0,1,0
226,0,1,0,0


In [18]:
dummies.describe()

,ecog_0,ecog_1,ecog_2,ecog_3
count,227.000000,227.000000,227.000000,227.000000
mean,0.277533,0.497797,0.220264,0.004405
std,0.448771,0.501100,0.415341,0.066372
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [19]:
#solamente nos quedamos con 3 dummies
dummies=dummies[['ecog_1','ecog_2']]
#dejamos fuera ecog_0

In [20]:
## Unir dummies con df cancer
df_cancer=pd.concat([df_cancer,dummies],axis=1)

In [21]:
df_cancer

,time,status,age,sex,ph.ecog,ph.karno,pat.karno,meal.cal,wt.loss,ecog_1,ecog_2
0,306,1,74,1,1,100.0,100.0,1175.000000,9.831776,1,0
1,455,1,68,1,0,90.0,90.0,1225.000000,15.000000,0,0
2,1010,0,56,1,0,90.0,90.0,928.779006,15.000000,0,0
3,210,1,57,1,1,60.0,60.0,1150.000000,11.000000,1,0
4,883,1,60,1,0,90.0,90.0,928.779006,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...
223,188,0,77,1,1,60.0,60.0,928.779006,3.000000,1,0
224,191,0,39,1,0,90.0,90.0,2350.000000,-5.000000,0,0
225,105,0,75,2,2,70.0,70.0,1025.000000,5.000000,0,1
226,174,0,66,1,1,100.0,100.0,1075.000000,1.000000,1,0


In [23]:
df_cancer.drop('ph.ecog',axis=1)

,time,status,age,sex,ph.karno,pat.karno,meal.cal,wt.loss,ecog_1,ecog_2
0,306,1,74,1,100.0,100.0,1175.000000,9.831776,1,0
1,455,1,68,1,90.0,90.0,1225.000000,15.000000,0,0
2,1010,0,56,1,90.0,90.0,928.779006,15.000000,0,0
3,210,1,57,1,60.0,60.0,1150.000000,11.000000,1,0
4,883,1,60,1,90.0,90.0,928.779006,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...
223,188,0,77,1,60.0,60.0,928.779006,3.000000,1,0
224,191,0,39,1,90.0,90.0,2350.000000,-5.000000,0,0
225,105,0,75,2,70.0,70.0,1025.000000,5.000000,0,1
226,174,0,66,1,100.0,100.0,1075.000000,1.000000,1,0


In [24]:
##corroborar que sean numericas y que no haya valores perdidos
df_cancer.dtypes

time           int64
status         int64
age            int64
sex            int64
ph.ecog        int64
ph.karno     float64
pat.karno    float64
meal.cal     float64
wt.loss      float64
ecog_1         int32
ecog_2         int32
dtype: object

In [27]:
##Estadisticas descriptivas
df_cancer.describe()

,time,status,age,sex,ph.ecog,ph.karno,pat.karno,meal.cal,wt.loss,ecog_1,ecog_2
count,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000,227.000000
mean,306.264317,0.722467,62.458150,1.396476,0.951542,80.025617,79.999757,927.474067,9.734118,0.497797,0.220264
std,210.532764,0.448771,9.092045,0.490246,0.717872,14.544884,14.543192,358.375611,12.670492,0.501100,0.415341
min,5.000000,0.000000,39.000000,1.000000,0.000000,30.000000,30.000000,96.000000,-24.000000,0.000000,0.000000
25%,168.500000,0.000000,56.000000,1.000000,0.000000,70.000000,70.000000,768.000000,0.000000,0.000000,0.000000
50%,259.000000,1.000000,63.000000,1.000000,1.000000,80.000000,80.000000,928.779006,8.000000,0.000000,0.000000
75%,399.000000,1.000000,69.000000,2.000000,1.000000,90.000000,90.000000,1075.000000,15.000000,1.000000,0.000000
max,1022.000000,1.000000,82.000000,2.000000,3.000000,100.000000,100.000000,2600.000000,68.000000,1.000000,1.000000


In [32]:
df_cancer['status'].value_counts()

status
1    164
0     63
Name: count, dtype: int64

El 72% de los datos han sufrido el evento (morir), 39.6% son mujeres, el promedio de edad es 62.4, la edad más grande es 82 y la más joven es 39, dado que la media de pat.karno es 79.99 y la de ph.karno es de 82.03, lo que significa que en promedio los pacientes evaluan su estado de salud peor de lo que estan, el promedio de wt.loss es 9.73.

In [36]:
cph1=CoxPHFitter().fit(df_cancer,'time','status')
cph1.print_summary()

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\lifelines\fitters\coxph_fitter.py:1611: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
  warnings.warn(


<lifelines.CoxPHFitter: fitted with 227 total observations, 63 right-censored observations>
             duration col = 'time'
                event col = 'status'
      baseline estimation = breslow
   number of observations = 227
number of events observed = 164
   partial log-likelihood = -740.11
         time fit was run = 2023-09-18 20:55:46 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.00       1.00       0.01            -0.02             0.02                 0.98                 1.02
sex        -0.06       0.95       0.16            -0.37             0.26                 0.69                 1.30
ph.ecog     0.17       1.18       0.63            -1.07             1.41                 0.34                 4.09
ph.karno    0.15       1.16       0.52            -0.87             1.18                 0.42                 3.24
pat.karno  -0.15       0.86       0.52            -1.18             0.87                 0.31                 2.40
meal.cal    0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
wt.loss    -0.00       1.00       0.01            -0.01             0.01                 0.99                 1.01
ecog_1     -0.13       0.88       0.65            -1.40             1.14                 0.25                 3.12
ecog_2     -0.25       0.78       1.28            -2.76             2.25                 0.06                 9.48

            cmp to     z    p   -log2(p)
covariate                               
age           0.00  0.09 0.93       0.11
sex           0.00 -0.35 0.73       0.45
ph.ecog       0.00  0.26 0.79       0.34
ph.karno      0.00  0.29 0.77       0.37
pat.karno     0.00 -0.29 0.77       0.37
meal.cal      0.00  0.00 1.00       0.01
wt.loss       0.00 -0.17 0.86       0.21
ecog_1        0.00 -0.20 0.84       0.25
ecog_2        0.00 -0.20 0.84       0.25
---
Concordance = 0.66
Partial AIC = 1498.21
log-likelihood ratio test = 8.75 on 9 df
-log2(p) of ll-ratio test = 1.12

*Age* No se rechaza H0, age no afecta sobre el riesgo de morir de cancer

*Sex* Se rechaza H0, ser mujer disminuye el riesgo de morir de cancer en 44% respecto a los hombres

*Ph.karno* No se rechaza H0, no tiene efecto sobre el riesgo de morir de cancer

*Pat.karno* No se rechaza H0, no tiene efecto sobre el riesgo de morir de cancer

*meal.cal* No se rechaza H0, no tiene efecto sobre el riesgo de morir de cancer

*wt.loss* Se rechaza H0, el perder peso disminuye el riesgo de cancer en un 1%

In [37]:
(1-0.56)*100

43.99999999999999

Esta construido sobre el supuesto de riesgo proporciona constante

In [45]:
cph1.check_assumptions(df_cancer, p_value_threshold=0.05)
## Ho: Se viola el supuesto
## Ha: Se cumple el supuesto

Proportional hazard assumption looks okay.


[]

(No deberia de cumplir)

Como no se cumple el supuesto, debemos utilizar un Modelo Alternativo, el modelo de riesgo acelerado AFT

Primero debemos identificar si es Exponencial, Weibull o LogNormal

In [41]:
from lifelines import ExponentialFitter, WeibullFitter, LogNormalFitter

In [43]:
modeloexpo= ExponentialFitter().fit(df_cancer['time'],df_cancer['status'])
modeloweibull=WeibullFitter().fit(df_cancer['time'],df_cancer['status'])
logn=LogNormalFitter().fit(df_cancer['time'],df_cancer['status'])

In [44]:
modeloexpo.AIC_, modeloweibull.AIC_, logn.AIC_

(2314.2465289315023, 2298.8561137465936, 2330.351968846968)

In [49]:
## Elegimos el modelo weibull
from lifelines import WeibullAFTFitter
weibullAFT=WeibullAFTFitter().fit(df_cancer,'time','status')
weibullAFT.print_summary()

<lifelines.WeibullAFTFitter: fitted with 227 total observations, 63 right-censored observations>
             duration col = 'time'
                event col = 'status'
   number of observations = 227
number of events observed = 164
           log-likelihood = -1128.54
         time fit was run = 2023-09-18 21:45:32 UTC

---
                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param   covariate                                                                                                         
lambda_ age        -0.01       0.99       0.01            -0.02             0.01                 0.98                 1.01
        ecog_1      0.16       1.17       0.26            -0.36             0.67                 0.70                 1.96
        ecog_2      0.36       1.44       0.50            -0.61             1.34                 0.54                 3.82
        meal.cal   -0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
        pat.karno   0.48       1.62       0.22             0.04             0.92                 1.04                 2.52
        ph.ecog    -0.44       0.65       0.25            -0.92             0.04                 0.40                 1.05
        ph.karno   -0.47       0.62       0.22            -0.91            -0.03                 0.40                 0.97
        sex         0.40       1.49       0.12             0.16             0.64                 1.17                 1.90
        wt.loss     0.01       1.01       0.00            -0.00             0.02                 1.00                 1.02
        Intercept   5.41     223.56       0.68             4.09             6.73                59.54               839.47
rho_    Intercept   0.33       1.40       0.06             0.21             0.45                 1.24                 1.57

                    cmp to     z      p   -log2(p)
param   covariate                                 
lambda_ age           0.00 -1.07   0.28       1.82
        ecog_1        0.00  0.59   0.56       0.85
        ecog_2        0.00  0.73   0.46       1.10
        meal.cal      0.00 -0.16   0.88       0.19
        pat.karno     0.00  2.15   0.03       4.97
        ph.ecog       0.00 -1.78   0.08       3.73
        ph.karno      0.00 -2.12   0.03       4.86
        sex           0.00  3.23 <0.005       9.65
        wt.loss       0.00  1.69   0.09       3.47
        Intercept     0.00  8.01 <0.005      49.67
rho_    Intercept     0.00  5.43 <0.005      24.11
---
Concordance = 0.66
AIC = 2279.09
log-likelihood ratio test = 37.77 on 9 df
-log2(p) of ll-ratio test = 15.67

Age no tiene efecto sobre la medicina del tiempo de supervivencia